In [1]:
import pandas as pd
import numpy as np
from time import time, sleep
import colour
%matplotlib inline
key = open("c:\\users\cglan\\desktop\\key.txt","r").read()

In [26]:
map_data.head()

,latitude,longitude,name,grade,stars,starVotes,difficulty,tooltip,color_x,color_y,color
0,44.365712,-121.142434,"Squaw Rock, Spiral",5.1,4.0,8,5.0 to 5.3,"Squaw Rock, Spiral 5.1",#67a9cf,green,LimeGreen
1,44.369920,-121.122902,North-East Sramble,5.0,3.5,2,5.0 to 5.3,North-East Sramble 5.0,#67a9cf,green,LimeGreen
2,44.374193,-121.129838,Northwest Corner,5.1,3.3,3,5.0 to 5.3,Northwest Corner 5.1,#67a9cf,green,LimeGreen
3,44.365820,-121.142885,"The Sapling, Tilted Slab",5.3,3.0,2,5.0 to 5.3,"The Sapling, Tilted Slab 5.3",#67a9cf,green,LimeGreen
4,44.369933,-121.139132,Super Slab,5.6,4.1,182,5.3 to 5.6,Super Slab 5.6,#3f5ecd,#50a000,#2ad1c9


In [27]:
#length of degree
def lod(deg, kind):
    length_at_equator = 110.5742727
    
    if kind == 'lon':
        out = np.cos(deg * (2 * np.pi) / 360) * length_at_equator
    elif kind == 'lat':
        out = length_at_equator
    return out
        
type(lod(deg = -72.99768, kind = 'lat'))

def jitter(lon, lat, km):
    lon_km_per_degree = lod(lat,'lon')
    lon_degree_per_km = 1/lon_km_per_degree
    lon_rand = np.random.uniform(-1,1,1)
    lon_j = lon + lon_rand*lon_degree_per_km*km
    
    lat_km_per_degree = lod(lat,'lat')
    lat_degree_per_km = 1/lat_km_per_degree
    lat_rand = np.random.uniform(-1,1,1)
    lat_j = lat + lat_rand*(lat_degree_per_km)*km
    
    return [lat_j[0],lon_j[0]]
jitter(lon =  -72.99768, lat = 43, km = .001)    

def grabber(lat = "44.3657", long = "-121.143", dist = "20",
    mindiff = "5.0", maxdiff = "5.12", results = "500", key = key):
    send = "https://www.mountainproject.com/data/"
    send += "get-routes-for-lat-lon"
    send += "?lat=" + lat
    send += "&lon=" + long
    send += "&maxDistance="+dist
    send += "&minDiff="+mindiff
    send += "&maxDiff="+maxdiff
    send += "&maxResults="+results
    send += "&key=" + key
    read_json = pd.read_json(send, orient='columns')
    
    sleep(2)
    return pd.read_json(read_json['routes'].to_json(), orient= 'index')

def array_splitter(k, grade_max):
    grades = ['5.' + np.str(i) for i in np.arange(0,grade_max,1)]
    if (len(grades)/2) >=k: 
        grades_split = np.array_split(grades,k)
        grade_index = np.cumsum([len(i) for i in grades_split])
        objs =[]
        for item in grade_index:
            mindiff = grades[item - np.min(grade_index)]
            try:
                objs.append(
                    [mindiff, grades[item-np.sum((k//2)*2 != k)]]
                )
            except:
                objs.append(
                    [mindiff,
                     grades[len(grades)-1]
                    ])
                continue
    else: 
        objs = 'hi'
    return objs

In [13]:
cuts = array_splitter(5, 16)
route_json = [grabber(mindiff = grade[0], maxdiff=grade[1]) for grade in cuts]

add difficulty

In [28]:
route_list = route_json
for i in np.arange(0,len(route_list)):
    route_list[i]['difficulty'] = (" to ".join(cuts[i]))

In [29]:
route_data = pd.concat(route_list)
route_data = route_data.drop_duplicates('name')
route_data['grade'] = route_data['rating'].str.extract('(5.[0-9]{1,2})', expand = False)
route_data['risk'] = route_data['rating'].str.extract('(X|R|PG13)', expand = False)

In [30]:
map_data = route_data.reset_index(drop = True)
map_data = map_data[['latitude', 'longitude','name','grade','stars','starVotes','difficulty']]
map_data['tooltip'] = map_data['name'] + " " +map_data['grade'] # + " (" + to_map["difficulty"] + "; " + to_map["stars"].astype(str) + ")"
map_data.head()

,latitude,longitude,name,grade,stars,starVotes,difficulty,tooltip
0,44.3657,-121.1430,"Squaw Rock, Spiral",5.1,4.0,8,5.0 to 5.3,"Squaw Rock, Spiral 5.1"
1,44.3699,-121.1234,North-East Sramble,5.0,3.5,2,5.0 to 5.3,North-East Sramble 5.0
2,44.3744,-121.1294,Northwest Corner,5.1,3.3,3,5.0 to 5.3,Northwest Corner 5.1
3,44.3657,-121.1430,"The Sapling, Tilted Slab",5.3,3.0,2,5.0 to 5.3,"The Sapling, Tilted Slab 5.3"
4,44.3699,-121.1392,Super Slab,5.6,4.1,182,5.3 to 5.6,Super Slab 5.6


In [31]:
locationlist = map_data[['latitude', 'longitude']].values.tolist()
centroid = [np.sum([cord[0] for cord in locationlist])/len(locationlist), np.sum([cord[1] for cord in locationlist])/len(locationlist)]

In [32]:
map_data[['latitude','longitude']]= map_data[['latitude','longitude']].apply(lambda x: jitter(x[1],x[0], km = .05), axis =1)

In [33]:
red = colour.Color("#32CD32")
blue = colour.Color("#DC143C")
col_range = red.range_to(blue,len(map_data.difficulty.unique()))
col_list = pd.DataFrame(
    {"difficulty": map_data.difficulty.unique(),
     "color":[str(i) for i in col_range]})
map_data = map_data.merge(col_list, on = 'difficulty', how = 'inner')


In [34]:
import folium
mapit = folium.Map( location=centroid, zoom_start=15)
for point in range(0, len(map_data)):
    folium.CircleMarker(map_data.loc[point,['latitude','longitude']],
                    radius=1 + map_data.loc[point,'stars'],
                    popup=folium.Popup(map_data.loc[point,'tooltip'], parse_html=True),
                    color=map_data.loc[point,'color'],
                    fill_color='red',
                   ).add_to(mapit)

mapit.save('map3.html')